In [1]:
from LossFun import VIC

In [2]:
import torch

In [3]:
out_dim = 224
local_crops_number = 2
warmup_teacher_temp = 0.014
teacher_temp = 0.04
warmup_teacher_temp_epochs = 30
epochs = 100

In [4]:
# # ============ preparing loss ... ============
# dino_loss = VIC.DINOLoss(
#     out_dim,
#     local_crops_number + 2,  # total number of crops = 2 global crops + local_crops_number
#     warmup_teacher_temp,
#     teacher_temp,
#     warmup_teacher_temp_epochs,
#     epochs,
# )

In [5]:
# x = torch.rand(10, 224)
# y = torch.rand(2, 224)

In [6]:
# loss = dino_loss(x, y, epoch = 1)

In [7]:
# loss

In [8]:
# from types import SimpleNamespace

# # Example dictionary
# my_dict = {
#     'name': 'Alice',
#     'age': 30,
#     'city': 'New York'
# }

# # Convert dictionary to SimpleNamespace
# namespace = SimpleNamespace(**my_dict)

# # Accessing attributes
# print(namespace.name)  # Output: Alice
# print(namespace.age)   # Output: 30
# print(namespace.city)  # Output: New York


In [9]:
# import torch
# import torch.nn.functional as F

# # Example input tensor with shape (96, 3, 96, 96)
# input_tensor = torch.randn(96, 3, 96, 96, requires_grad=True)

# # Desired output size
# output_size = (224, 224)

# # Upsample the input tensor
# upsampled_tensor = F.interpolate(input_tensor, size=output_size, mode='bilinear', align_corners=False)

# # Define a simple loss function (mean of the tensor)
# loss = upsampled_tensor.mean()

# # Backpropagate the loss
# loss.backward()

# # Check if the gradients are not None
# print(input_tensor.grad is not None)  # Should print True


In [10]:
import os
from PIL import Image
from helpers import *
import matplotlib.pyplot as plt
import utils

In [11]:
import torch
import torch.nn as nn
from TUVW import UViT
from UModels.DiT import DiT_Anomaly
from GaussianDiffusion import GaussianDiffusionModel, get_beta_schedule
from torchvision import transforms as pth_transforms

attention mode is math


In [24]:
model1 = UViT(img_size = 224, patch_size=16, in_chans=3, embed_dim = 768,
             depth=12, num_heads=12, mlp_ratio=4., qkv_bias=False, qk_scale=None, norm_layer=nn.LayerNorm, 
             mlp_time_embed=False, num_classes=None,
             use_checkpoint=False, conv=True, skip=True)
model = DiT_Anomaly(
    input_size=256,
    num_classes=2)

In [13]:
multi_model = utils.MultiCropWrapper(model)

In [14]:
# model_path = "model/diff-params-ARGS=73/params-final.pt"
model_path = "saving_dir/Brain_MRI/UViT/checkpoint.pth"
model_path = r"C:\Users\Admin\Documents\Anomaly Detection\AnoDDPM\model\diff-params-ARGS=200\params-final.pt"

In [15]:
wts = torch.load(model_path)

In [16]:
args = {}

In [17]:
model.load_state_dict(wts['ema'])
# multi_model.load_state_dict(wts['teacher'])


<All keys matched successfully>

In [18]:
betas = get_beta_schedule(1000, 'cosine')        
diffusion = GaussianDiffusionModel(
    256,
    betas, 
    args= args
)       

model.eval()
model.to('cuda')

DiT(
  (x_embedder): PatchEmbed(
    (proj): Conv2d(1, 768, kernel_size=(16, 16), stride=(16, 16))
    (norm): Identity()
  )
  (t_embedder): TimestepEmbedder(
    (mlp): Sequential(
      (0): Linear(in_features=256, out_features=768, bias=True)
      (1): SiLU()
      (2): Linear(in_features=768, out_features=768, bias=True)
    )
  )
  (y_embedder): LabelEmbedder(
    (embedding_table): Embedding(3, 768)
  )
  (blocks): ModuleList(
    (0-11): 12 x DiTBlock(
      (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=False)
      (attn): Attention(
        (qkv): Linear(in_features=768, out_features=2304, bias=True)
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=768, out_features=768, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=False)
      (mlp): Mlp(
        (fc1): Linear(in_features=768, out_features=3072, bias=True)
        (act): GELU(approximate='t

In [19]:
image_path = "DATASETS/brainMRI/train/good/No22.png"

In [20]:
if os.path.isfile(image_path):
    with open(image_path, 'rb') as f:
        img = Image.open(f)
        img = img.convert('L')
else:
    print(f"Provided image path {args.image_path} is non valid.")
    sys.exit(1)
transform = pth_transforms.Compose([
    pth_transforms.Resize((256, 256)),
    pth_transforms.ToTensor(),
#     pth_transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
])
img = transform(img).unsqueeze(0)


In [21]:
lab = torch.tensor([0]).cuda()

In [ ]:
multi_model

In [25]:
for n, p in model.named_parameters():
    print(n)
#     if "last_layer" in n:
#         p.grad = None

pos_embed
x_embedder.proj.weight
x_embedder.proj.bias
t_embedder.mlp.0.weight
t_embedder.mlp.0.bias
t_embedder.mlp.2.weight
t_embedder.mlp.2.bias
y_embedder.embedding_table.weight
blocks.0.attn.qkv.weight
blocks.0.attn.qkv.bias
blocks.0.attn.proj.weight
blocks.0.attn.proj.bias
blocks.0.mlp.fc1.weight
blocks.0.mlp.fc1.bias
blocks.0.mlp.fc2.weight
blocks.0.mlp.fc2.bias
blocks.0.adaLN_modulation.1.weight
blocks.0.adaLN_modulation.1.bias
blocks.1.attn.qkv.weight
blocks.1.attn.qkv.bias
blocks.1.attn.proj.weight
blocks.1.attn.proj.bias
blocks.1.mlp.fc1.weight
blocks.1.mlp.fc1.bias
blocks.1.mlp.fc2.weight
blocks.1.mlp.fc2.bias
blocks.1.adaLN_modulation.1.weight
blocks.1.adaLN_modulation.1.bias
blocks.2.attn.qkv.weight
blocks.2.attn.qkv.bias
blocks.2.attn.proj.weight
blocks.2.attn.proj.bias
blocks.2.mlp.fc1.weight
blocks.2.mlp.fc1.bias
blocks.2.mlp.fc2.weight
blocks.2.mlp.fc2.bias
blocks.2.adaLN_modulation.1.weight
blocks.2.adaLN_modulation.1.bias
blocks.3.attn.qkv.weight
blocks.3.attn.qkv.bia

In [22]:
samples = diffusion.forward_backward(
    model,
    img.to('cuda'),
    lab,
    see_whole_sequence=None,
    t_distance=1000, denoise_fn = 'gauss'
)
outputs = torch.cat(
    (
        img.to('cuda'), 
        samples,                    
    )
) 
plt.imshow(gridify_output(outputs, 7)[..., 0], cmap = 'gray')

TypeError: only integer tensors of a single element can be converted to an index